src: <https://almondine-song-c43.notion.site/Homework-3-ec8534279e2045479353509d5c3c73a1>

Use this resource as a reference: https://cryptobook.nakov.com/digital-signatures/ecdsa-sign-verify-messages

The following may also be helpful:
- https://www.rareskills.io/post/finite-fields
- https://www.rareskills.io/post/elliptic-curve-addition
- https://www.rareskills.io/post/elliptic-curves-finite-fields

Implement ECDSA from scratch. You want to use the secp256k1 curve. See here for a reference library:  
<https://www.rareskills.io/post/generate-ethereum-address-from-private-key-python>

1. pick a private key
2. generate the public key using that private key (not the eth address, the public key)
3. pick a message **m** and hash it to produce **h** (h can be though of as a 256 bit number)
4. sign **m** using your private key and a randomly chosen nonce k. produce (r, s, h, PubKey)
5. verify (r, s, h, PubKey) is valid

You may use a library for point multiplication, but everything else you must do from scratch.

Pay close attention to the distinction between the curve order and the prime number $p$ we compute the modulus of $y^2=x^3+b \pmod p$.

In [ ]:
from ecpy.curves import Curve
from ecpy.keys import ECPublicKey, ECPrivateKey
from sha3 import keccak_256

sk = 0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80
cv = Curve.get_curve('secp256k1')
pv_key = ECPrivateKey(sk, cv); print(f"pv_key: {pv_key}")
pk = pv_key.get_public_key(); print(f"pk: {pk}")
self_calc_pk = sk * cv.generator; print(f"self_calc_pk: {self_calc_pk}")

x_part = pk.W.x.to_bytes(32, byteorder='big'); print(f"x_part: {list(x_part)}")
y_part = pk.W.y.to_bytes(32, byteorder='big'); print(f"y_part: {list(y_part)}")
concat_x_y = x_part + y_part; print(f"concat: {concat_x_y}")
eth_addr = '0x' + keccak_256(concat_x_y).digest()[-20:].hex(); print(f"eth addr: {eth_addr}")

In [ ]:
from ecpy.curves import Curve
from sha3 import keccak_256
import secrets

# https://cryptobook.nakov.com/digital-signatures/ecdsa-sign-verify-messages

class Ecdsa:
    cv = Curve.get_curve("secp256k1")
    
    def __init__(self, sk=None):
        if sk == None:
            sk = secrets.randbelow(2**256)
        self.sk = sk

    def public_key(self):
        if hasattr(self, "pk") and self.pk != None:
            return self.pk
        self.pk = self.sk * cv.generator
        return self.pk

    def sign(self, message):
        hsh = keccak_256(str.encode(message))
        k = secrets.randbelow(cv.field); print(f"field: {cv.field}")
        R = k * cv.generator
        r = R.x; print(f"r: {r}")
        s = pow(k, -1, cv.field) * (hsh + r * self.sk) % cv.field
        # return (r, s)

    @classmethod
    def verify(cls):
        pass

    @classmethod
    def debug(cls):
        print(cls.cv.generator)

    # Can you verify the secp256k1 curve order and generator point in terms of number
    # Now, how do you enforce type in Python? For example, what is being passed in keccak_256()?

def main():
    sk = 43765866871480949447442477281291151637608214855806437531566966568646888625943
    # sk = None
    ecdsa = Ecdsa(sk)
    print(Ecdsa.debug())
    print(ecdsa.sign("abcdef"))


main()